# MCMC

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install libraries

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Para a maior parte deste livro temos utilizado métodos de grelha para aproximar as distribuições posteriores.

Para modelos com um ou dois parâmetros, os algoritmos de grelha são rápidos e os resultados são suficientemente precisos para a maioria dos fins práticos.

Com três parâmetros, começam a ser lentos, e com mais de três não são normalmente práticos.

No capítulo anterior vimos que podemos resolver alguns problemas usando os priores conjugados.

Mas os problemas que podemos resolver desta forma tendem a ser os mesmos que podemos resolver com algoritmos de grelha.

Para problemas com mais do que alguns parâmetros, a ferramenta mais poderosa que temos é MCMC, que significa "Markov chain Monte Carlo".

Neste contexto, "Monte Carlo" refere-se a métodos que geram amostras aleatórias a partir de uma distribuição.

Ao contrário dos métodos de grelha, os métodos MCMC não tentam calcular a distribuição posterior; em vez disso, recolhem amostras a partir dela.

Pode parecer estranho que se possa gerar uma amostra sem nunca calcular a distribuição, mas essa é a magia do MCMC.

Para demonstrar, vamos começar por resolver o problema do Campeonato do Mundo.

Sim, mais uma vez.



## O problema do Campeonato do Mundo

Em <<_PoissonProcessos>> modelamos a marcação de golos no futebol (futebol) como um processo Poisson caracterizado por uma taxa de marcação de golos, denotado $\lambda$.

Utilizámos uma distribuição gama para representar a distribuição prévia de $\lambda$, depois utilizámos o resultado do jogo para calcular a distribuição posterior para ambas as equipas.

Para responder à primeira pergunta, utilizámos as distribuições posteriores para calcular a "probabilidade de superioridade" para a França.

Para responder à segunda pergunta, calculámos as distribuições preditivas posteriores para cada equipa, ou seja, a distribuição dos objectivos que esperamos numa desforra.

Neste capítulo vamos resolver este problema novamente usando PyMC3, que é uma biblioteca que fornece implementações de vários métodos MCMC.

Mas vamos começar por rever a aproximação da grelha da distribuição prévia e da distribuição preditiva prévia.

## Aproximação da grelha

Como fizemos em <<_TheGammaDistribution>> usaremos uma distribuição gama com o parâmetro $\alpha=1,4$ para representar o anterior.

In [4]:
from scipy.stats import gamma

alpha = 1.4
prior_dist = gamma(alpha)

Vou utilizar `linspace` para gerar valores possíveis por $\lambda$, e `pmf_from_dist` para calcular uma aproximação discreta do anterior.

In [5]:
import numpy as np
from utils import pmf_from_dist

lams = np.linspace(0, 10, 101)
prior_pmf = pmf_from_dist(prior_dist, lams)

Podemos utilizar a distribuição de Poisson para calcular a probabilidade dos dados; como exemplo, utilizaremos 4 objectivos.

In [6]:
from scipy.stats import poisson

data = 4
likelihood = poisson.pmf(data, lams)

Agora podemos fazer a actualização da forma habitual.

In [7]:
posterior = prior_pmf * likelihood
posterior.normalize()

Em breve resolveremos o mesmo problema com PyMC3, mas primeiro será útil introduzir algo novo: a distribuição preditiva prévia.

## Distribuição Preditiva Preditiva

Vimos a distribuição preditiva posterior nos capítulos anteriores; a distribuição preditiva anterior é semelhante, excepto que (como já deve ter adivinhado) se baseia na anterior.

Para estimar a distribuição preditiva anterior, começaremos por retirar uma amostra do anterior.

In [8]:
sample_prior = prior_dist.rvs(1000)

O resultado é um conjunto de valores possíveis para a taxa de pontuação, $\lambda$.

Para cada valor em `sample_prior`, irei gerar um valor a partir de uma distribuição de Poisson.

In [9]:
from scipy.stats import poisson

sample_prior_pred = poisson.rvs(sample_prior)

A "amostra_prior_pred" é uma amostra da distribuição preditiva anterior.

Para ver o que parece, vamos calcular o PMF da amostra.

In [10]:
from empiricaldist import Pmf

pmf_prior_pred = Pmf.from_seq(sample_prior_pred)

E aqui está o que parece:

In [11]:
from utils import decorate

pmf_prior_pred.bar()
decorate(xlabel='Number of goals',
         ylabel='PMF',
         title='Prior Predictive Distribution')

Uma razão para calcular a distribuição preditiva prévia é verificar se o nosso modelo do sistema parece razoável.

Neste caso, a distribuição de golos parece consistente com o que sabemos sobre o Campeonato do Mundo de Futebol.

Mas neste capítulo temos outra razão: computar a distribuição preditiva prévia é um primeiro passo para a utilização do MCMC.

## Introduzindo PyMC3

PyMC3 é uma biblioteca Python que fornece vários métodos MCMC.

Para utilizar PyMC3, temos de especificar um modelo do processo que gera os dados.

Neste exemplo, o modelo tem duas etapas:

* Primeiro tiramos uma taxa de marcação de objectivos da distribuição prévia,

* Depois retiramos uma série de objectivos de uma distribuição de Poisson.

Eis como especificamos este modelo em PyMC3:

In [12]:
import pymc3 as pm

with pm.Model() as model:
    lam = pm.Gamma('lam', alpha=1.4, beta=1.0)
    goals = pm.Poisson('goals', lam)

Depois de importar `pymc3`, criamos um objecto `modelo` chamado `modelo`.

Se não estiver familiarizado com a declaração "com" em Python, é uma forma de associar um bloco de declarações a um objecto.

Neste exemplo, as duas declarações indentadas estão associadas ao novo objecto `Modelo`.  Como resultado, quando criamos os objectos de distribuição, "Gama" e "Poisson", eles são adicionados ao "Modelo".

Dentro da declaração "com":

* A primeira linha cria o anterior, que é uma distribuição gama com os parâmetros dados.

* A segunda linha cria a previsão prévia, que é uma distribuição de Poisson com o parâmetro `lam`.

O primeiro parâmetro de `Gamma` e `Poisson` é um nome de variável de cadeia.

PyMC3 fornece uma função que gera uma representação visual do modelo.

In [13]:
pm.model_to_graphviz(model)

Nesta visualização, as ovais mostram que o "lam" é retirado de uma distribuição gama e o "goals" é retirado de uma distribuição Poisson.

A seta mostra que os valores de `lam` são utilizados como parâmetros para a distribuição de `goal'.

## Amostragem do Prior

PyMC3 fornece uma função que gera amostras a partir das distribuições preditivas anteriores e anteriores.

Podemos utilizar uma declaração "com" para executar esta função no contexto do modelo.

In [14]:
with model:
    trace = pm.sample_prior_predictive(1000)

O resultado é um objecto semelhante a um dicionário que mapeia desde as variáveis, `lam` e `goals', até às amostras.

Podemos extrair a amostra de `lam` desta forma:

In [15]:
sample_prior_pymc = trace['lam']
sample_prior_pymc.shape

A figura seguinte compara o CDF desta amostra com o CDF da amostra que gerámos utilizando o objecto `gamma` da SciPy.

In [16]:
from empiricaldist import Cdf

def plot_cdf(sample, **options):
    """Plot the CDF of a sample.
    
    sample: sequence of quantities
    """
    Cdf.from_seq(sample).plot(**options)

In [17]:
plot_cdf(sample_prior, 
         label='SciPy sample',
         color='C5')
plot_cdf(sample_prior_pymc, 
         label='PyMC3 sample',
         color='C0')
decorate(xlabel=r'Goals per game ($\lambda$)',
         ylabel='CDF',
         title='Prior distribution')

Os resultados são semelhantes, o que confirma que a especificação do modelo é correcta e que o amostrador funciona como anunciado.

A partir do traço podemos também extrair `goals', que é uma amostra da distribuição preditiva anterior.

In [18]:
sample_prior_pred_pymc = trace['goals']
sample_prior_pred_pymc.shape

E podemos compará-lo com a amostra que gerámos utilizando o objecto `poisson` da SciPy.

Como as quantidades na distribuição preditiva posterior são discretas (número de objectivos), vou traçar os CDFs como funções de passo.

In [19]:
def plot_pred(sample, **options):
    Cdf.from_seq(sample).step(**options)

In [20]:
plot_pred(sample_prior_pred, 
          label='SciPy sample', 
          color='C5')
plot_pred(sample_prior_pred_pymc, 
          label='PyMC3 sample', 
          color='C13')
decorate(xlabel='Number of goals',
         ylabel='PMF',
         title='Prior Predictive Distribution')

## Quando é que chegamos à Inferência?

Finalmente, estamos prontos para uma inferência real.  Só temos de fazer uma pequena mudança.

Aqui está o modelo que utilizámos para gerar a distribuição preditiva prévia:

In [21]:
with pm.Model() as model:
    lam = pm.Gamma('lam', alpha=1.4, beta=1.0)
    goals = pm.Poisson('goals', lam)

E aqui está o modelo que vamos utilizar para calcular a distribuição posterior.

In [22]:
with pm.Model() as model2:
    lam = pm.Gamma('lam', alpha=1.4, beta=1.0)
    goals = pm.Poisson('goals', lam, observed=4)

A diferença é que marcamos os objectivos como "observados" e fornecemos os dados observados, `4`.



In [23]:
options = dict(return_inferencedata=False)

with model2:
    trace2 = pm.sample(500, **options)

Embora a especificação destes modelos seja semelhante, o processo de amostragem é muito diferente.

Não vou entrar nos detalhes de como funciona PyMC3, mas aqui estão algumas coisas de que deve estar ciente:

* Dependendo do modelo, PyMC3 usa um dos vários métodos MCMC; neste exemplo, usa o [No U-Turn Sampler](https://en.wikipedia.org/wiki/Hamiltonian_Monte_Carlo#No_U-Turn_Sampler) (NUTS), que é um dos métodos mais eficientes e fiáveis que temos.

* Quando o amostrador começa, os primeiros valores que gera não são normalmente uma amostra representativa da distribuição posterior, pelo que estes valores são descartados.  Este processo é chamado "tuning".

* Em vez de usar uma única cadeia de Markov, PyMC3 usa múltiplas cadeias.  Depois podemos comparar resultados de múltiplas cadeias para nos certificarmos de que são consistentes.

Embora tenhamos pedido uma amostra de 500, PyMC3 gerou duas amostras de 1000, descartou metade de cada, e devolveu as restantes 1000.

A partir do `trace2` podemos extrair uma amostra da distribuição posterior, como esta:

In [24]:
sample_post_pymc = trace2['lam']

In [25]:
sample_post_pymc.shape

E podemos comparar o CDF desta amostra com o CDF posterior que calculámos por aproximação da grelha:

In [26]:
posterior.make_cdf().plot(label='posterior grid', 
                          color='C5')
plot_cdf(sample_post_pymc, 
         label='PyMC3 sample',
         color='C4')

decorate(xlabel=r'Goals per game ($\lambda$)',
         ylabel='CDF',
         title='Posterior distribution')

Os resultados de PyMC3 são consistentes com os resultados da aproximação da grelha.

## Distribuição Preditiva Posterior

Por fim, para colher amostras da distribuição preditiva posterior, podemos utilizar `samostra_posterior_predictiva':

In [27]:
with model2:
    post_pred = pm.sample_posterior_predictive(trace2)

O resultado é um dicionário que contém uma amostra de "objectivos".

In [28]:
sample_post_pred_pymc = post_pred['goals']

In [29]:
sample_post_pred_pymc.shape

Também vou gerar uma amostra a partir da distribuição posterior que calculámos por aproximação da grelha.

In [30]:
sample_post = posterior.sample(1000)
sample_post_pred = poisson(sample_post).rvs()

E podemos comparar as duas amostras.

In [31]:
plot_pred(sample_post_pred, 
          label='grid sample',
          color='C5')
plot_pred(sample_post_pred_pymc, 
          label='PyMC3 sample',
          color='C12')

decorate(xlabel='Number of goals',
         ylabel='PMF',
         title='Posterior Predictive Distribution')

Mais uma vez, os resultados são consistentes.

Assim, estabelecemos que podemos calcular os mesmos resultados utilizando uma aproximação da grelha ou PyMC3.

Mas pode não ser claro porquê.

Neste exemplo, o algoritmo da grelha requer menos cálculos do que o MCMC, e o resultado é uma boa aproximação da distribuição posterior, em vez de uma amostra.

No entanto, este é um modelo simples com apenas um parâmetro.

De facto, poderíamos tê-lo resolvido com ainda menos cálculo, utilizando um conjugado prévio.

O poder do PyMC3 será mais claro com um modelo mais complexo.

## Felicidade

Recentemente li ["Happiness and Life Satisfaction"](https://ourworldindata.org/happiness-and-life-satisfaction)

por Esteban Ortiz-Ospina e Max Roser, que discute (entre muitas outras coisas) a relação entre rendimento e felicidade, tanto entre países, dentro dos países, como ao longo do tempo.

Cita o ["World Happiness Report"](https://worldhappiness.report/), que inclui [results of a multiple regression analysis](https://worldhappiness.report/ed/2020/social-environments-for-world-happiness/) que explora a relação entre a felicidade e seis factores potencialmente preditivos:

* Rendimento representado pelo PIB per capita

* Apoio social

* Esperança de vida saudável à nascença

* Liberdade para fazer escolhas de vida

* Generosidade

* Percepções de corrupção



A variável dependente é a média nacional de respostas à "pergunta da escada de Cantril" utilizada pela [Gallup World Poll](https://news.gallup.com/poll/122453/understanding-gallup-uses-cantril-scale.aspx):

> Por favor, imagine uma escada com degraus numerados de zero na parte inferior a 10 na parte superior. O topo da escada representa a melhor vida possível para si e o fundo da escada representa a pior vida possível para si. Em que degrau da escada diria que se sente pessoalmente de pé neste momento?

Vou referir-me às respostas como "felicidade", mas poderia ser mais preciso pensar nelas como uma medida de satisfação com a qualidade de vida.

Nas próximas secções replicaremos a análise neste relatório usando a regressão Bayesiana.

Os dados deste relatório podem ser [downloaded from here](https://happiness-report.s3.amazonaws.com/2020/WHR20_DataForFigure2.1.xls).

In [32]:
# Get the data file

download('https://happiness-report.s3.amazonaws.com/2020/WHR20_DataForFigure2.1.xls')

Podemos utilizar Pandas para ler os dados num "DataFrame".

In [33]:
import pandas as pd

filename = 'WHR20_DataForFigure2.1.xls'
df = pd.read_excel(filename)

In [34]:
df.head(3)

In [35]:
df.shape

O "DataFrame" tem uma linha para cada um de 153 países e uma coluna para cada uma de 20 variáveis.

A coluna chamada "pontuação da escada" contém as medidas de felicidade que vamos tentar prever.

In [36]:
score = df['Ladder score']

## Regressão simples

Para começar, vejamos a relação entre a felicidade e o rendimento, representado pelo produto interno bruto (PIB) por pessoa.

A coluna denominada "PIB per capita registado" representa o logaritmo natural do PIB para cada país, dividido pela população, corrigido por [purchasing power parity](https://en.wikipedia.org/wiki/Purchasing_power_parity) (PPP).

In [37]:
log_gdp = df['Logged GDP per capita']

A figura seguinte é um gráfico de "score" versus "log_gdp", com um marcador para cada país.

In [38]:
import matplotlib.pyplot as plt

plt.plot(log_gdp, score, '.')

decorate(xlabel='Log GDP per capita at PPP',
         ylabel='Happiness ladder score')

É evidente que existe uma relação entre estas variáveis: as pessoas nos países com PIB mais elevado reportam geralmente níveis de felicidade mais elevados.

Podemos utilizar o `linregress` de SciPy para calcular uma regressão simples destas variáveis.

In [39]:
from scipy.stats import linregress

result = linregress(log_gdp, score)

E aqui estão os resultados.

In [40]:
pd.DataFrame([result.slope, result.intercept],
             index=['Slope', 'Intercept'],
             columns=[''])

A inclinação estimada é de cerca de 0,72, o que sugere que um aumento de uma unidade em log-GDP, que é um factor de $e \ aprox 2,7$ no PIB, está associado a um aumento de 0,72 unidades na escada da felicidade.

Agora vamos estimar os mesmos parâmetros usando o PyMC3.

Usaremos o mesmo modelo de regressão que na Secção <<_RegressãoModelo>>>:

$$y = a x + b + \epsilon$$

onde $y$ é a variável dependente (escada de pontuação), $x$ é a variável preditiva (log GDP) e $\silon$ é uma série de valores de uma distribuição normal com desvio padrão $\sigma$.

$a$ e $b$ são o declive e a intercepção da linha de regressão.

São parâmetros desconhecidos, pelo que utilizaremos os dados para os estimar.

A especificação PyMC3 deste modelo é a seguinte.

In [41]:
x_data = log_gdp
y_data = score

with pm.Model() as model3:
    a = pm.Uniform('a', 0, 4)
    b = pm.Uniform('b', -4, 4)
    sigma = pm.Uniform('sigma', 0, 2)

    y_est = a * x_data + b
    y = pm.Normal('y', 
                  mu=y_est, sd=sigma, 
                  observed=y_data)

As distribuições anteriores para os parâmetros `a`, `b`, e `sigma` são uniformes com gamas suficientemente amplas para cobrir as distribuições posteriores.

O valor estimado da variável dependente, com base na equação de regressão, é o valor estimado da variável dependente.

E `y` é uma distribuição normal com média `y_est` e desvio padrão `sigma`.

Notar como os dados são incluídos no modelo:

* Os valores da variável preditiva, `x_dados`, são utilizados para calcular `y_est`.

* Os valores da variável dependente, `y_data`, são fornecidos como os valores observados de `y`.

Agora podemos utilizar este modelo para gerar uma amostra a partir da distribuição posterior.

In [42]:
with model3:
    trace3 = pm.sample(500, **options)


Pode ignorá-los por agora.

O resultado é um objecto que contém amostras da distribuição posterior conjunta de `a`, `b`, e `sigma`.

In [43]:
trace3

ArviZ fornece `plot_posterior`, que podemos utilizar para traçar as distribuições posteriores dos parâmetros.

Aqui estão as distribuições posteriores de inclinação, `a`, e intercepção, `b`.

In [44]:
import arviz as az

with model3:
    az.plot_posterior(trace3, var_names=['a', 'b']);

Os gráficos mostram as distribuições das amostras, estimadas pelo KDE, e 94% de intervalos credíveis.  Na figura, "HDI" significa ["highest-density interval"](https://www.sciencedirect.com/topics/mathematics/highest-density-interval).

Os meios destas amostras são consistentes com os parâmetros que estimamos com `linregress'.

In [45]:
print('Sample mean:', trace3['a'].mean())
print('Regression slope:', result.slope)

In [46]:
print('Sample mean:', trace3['b'].mean())
print('Regression intercept:', result.intercept)

Finalmente, podemos verificar a distribuição marginal posterior do `sigma`.

In [47]:
az.plot_posterior(trace3['sigma']);

Os valores na distribuição posterior do `sigma` parecem plausíveis.

O modelo de regressão simples tem apenas três parâmetros, pelo que poderíamos ter utilizado um algoritmo de grelha.

Mas o modelo de regressão no relatório de felicidade tem seis variáveis preditivas, pelo que tem oito parâmetros no total, incluindo a intercepção e o `sigma`.

Não é prático calcular uma aproximação em grelha para um modelo com oito parâmetros.

Mesmo uma grelha grosseira, com 20 pontos ao longo de cada dimensão, teria mais de 25 mil milhões de pontos.

E com 153 países, teríamos de calcular quase 4 triliões de probabilidades.

Mas PyMC3 pode manusear confortavelmente um modelo com oito parâmetros, como veremos na secção seguinte.

In [48]:
20 ** 8 / 1e9

In [49]:
153 * 20 ** 8 / 1e12

## Regressão múltipla

Antes de implementarmos o modelo de regressão múltipla, irei seleccionar as colunas de que precisamos no `DataFrame'.

In [50]:
columns = ['Ladder score',
           'Logged GDP per capita',
           'Social support',
           'Healthy life expectancy',
           'Freedom to make life choices',
           'Generosity',
           'Perceptions of corruption']

subset = df[columns]

In [51]:
subset.head(3)

As variáveis preditivas têm unidades diferentes: log-GDP está em log-dólares, a esperança de vida está em anos, e as outras variáveis estão em escalas arbitrárias.

Para tornar estes factores comparáveis, padronizarei os dados de modo a que cada variável tenha a média 0 e o desvio padrão 1.

In [52]:
standardized = (subset - subset.mean()) / subset.std()

Agora vamos construir o modelo.

Vou extrair a variável dependente.

In [53]:
y_data = standardized['Ladder score']

E as variáveis dependentes.

In [54]:
x1 = standardized[columns[1]]
x2 = standardized[columns[2]]
x3 = standardized[columns[3]]
x4 = standardized[columns[4]]
x5 = standardized[columns[5]]
x6 = standardized[columns[6]]

E aqui está o modelo.  O "b0" é a intercepção; "b1" até "b6" são os parâmetros associados às variáveis preditivas.

In [55]:
with pm.Model() as model4:
    b0 = pm.Uniform('b0', -4, 4)
    b1 = pm.Uniform('b1', -4, 4)
    b2 = pm.Uniform('b2', -4, 4)
    b3 = pm.Uniform('b3', -4, 4)
    b4 = pm.Uniform('b4', -4, 4)
    b5 = pm.Uniform('b5', -4, 4)
    b6 = pm.Uniform('b6', -4, 4)
    sigma = pm.Uniform('sigma', 0, 2)

    y_est = b0 + b1*x1 + b2*x2 + b3*x3 + b4*x4 + b5*x5 + b6*x6
    y = pm.Normal('y', 
                  mu=y_est, sd=sigma, 
                  observed=y_data)

Poderíamos expressar este modelo de forma mais concisa utilizando um vector de variáveis preditivas e um vector de parâmetros, mas decidi mantê-lo simples.

Agora, podemos recolher amostras a partir da distribuição posterior conjunta.

In [56]:
with model4:
    trace4 = pm.sample(500, **options)

Como padronizámos os dados, esperamos que a intercepção seja 0, e de facto a média posterior de `b0` está próxima de 0.

In [57]:
trace4['b0'].mean()

Podemos também verificar a média posterior do `sigma`:

In [58]:
trace4['sigma'].mean()

In [59]:
param_names = ['b1', 'b3', 'b3', 'b4', 'b5', 'b6']

means = [trace4[name].mean() 
         for name in param_names]

Podemos também calcular 94% de intervalos credíveis (entre o 3º e o 97º percentis).

In [60]:
def credible_interval(sample):
    """Compute 94% credible interval."""
    ci = np.percentile(sample, [3, 97])
    return np.round(ci, 3)

cis = [credible_interval(trace4[name])
       for name in param_names]

O quadro seguinte resume os resultados.

In [61]:
index = columns[1:]
table = pd.DataFrame(index=index)
table['Posterior mean'] = np.round(means, 3)
table['94% CI'] = cis
table

Parece que o PIB tem a mais forte associação com a felicidade (ou satisfação), seguido de apoio social, esperança de vida e liberdade.

Depois de controlar para esses outros factores, os parâmetros dos outros factores são substancialmente menores, e uma vez que a IC para a generosidade inclui 0, é plausível que a generosidade não está substancialmente relacionada com a felicidade, pelo menos tal como foram medidos neste estudo.

Este exemplo demonstra o poder do MCMC para lidar com modelos com mais do que alguns parâmetros.

Mas não demonstra realmente o poder da regressão Bayesiana.

Se o objectivo de um modelo de regressão é estimar parâmetros, não há grande vantagem na regressão Bayesiana em comparação com a regressão convencional dos mínimos quadrados.

Os métodos Bayesianos são mais úteis se pretendemos utilizar a distribuição posterior dos parâmetros como parte de um processo de análise de decisão.

## Resumo

Neste capítulo utilizámos o PyMC3 para implementar dois modelos que já vimos anteriormente: um modelo Poisson de marcação de golos no futebol e um modelo simples de regressão.

Depois implementámos um modelo de regressão múltipla que não teria sido possível calcular com uma aproximação à grelha.

O MCMC é mais poderoso do que os métodos de rede, mas essa energia vem com algumas desvantagens:

* Os algoritmos MCMC são trabalhosos.  O mesmo modelo pode comportar-se bem com alguns antecedentes e menos bem com outros.  E o processo de amostragem produz frequentemente avisos sobre as etapas de afinação, divergências, "estatísticas de r-cavalo", taxas de aceitação, e amostras eficazes.  É necessária alguma perícia para diagnosticar e corrigir estas questões.

* Acho mais fácil desenvolver modelos gradualmente utilizando algoritmos de grelha, verificando resultados intermédios ao longo do percurso.  Com o PyMC3, não é tão fácil ter a certeza de ter especificado correctamente um modelo.

Por estas razões, recomendo um processo de desenvolvimento de modelos que comece com algoritmos de grelha e recorra ao MCMC, se necessário.

Como vimos nos capítulos anteriores, é possível resolver muitos problemas do mundo real com métodos de grelha.

Mas quando se precisa do MCMC, é útil ter um algoritmo de grelha para comparar (mesmo que se baseie num modelo mais simples).

Todos os modelos deste livro podem ser implementados em PyMC3, mas alguns deles são mais fáceis de traduzir do que outros.

Nos exercícios, terá uma oportunidade de praticar.

## Exercícios

**Exercício:** Como aquecimento, vamos usar o PyMC3 para resolver o problema do Euro.

Suponhamos que rodamos uma moeda 250 vezes e ela vem à cabeça 140 vezes.

Qual é a distribuição posterior de $x$, a probabilidade de cabeças?

Para o anterior, utilizar uma distribuição beta com parâmetros $\alpha=1$ e $\beta=1$.

Ver [the PyMC3 documentation](https://docs.pymc.io/api/distributions/continuous.html) para a lista de distribuições contínuas.

In [62]:
# Solution goes here

**Exercício:** Agora vamos usar PyMC3 para replicar a solução para o problema do Urso pardo em <<_TheGrizzlyBearProblem>>, que se baseia na distribuição hipergeométrica.

Vou apresentar o problema com uma notação ligeiramente diferente, para o tornar consistente com PyMC3.

Suponha que durante a primeira sessão, `k=23` ursos são etiquetados.  Durante a segunda sessão, `n=19` ursos são identificados, dos quais `x=4` foram etiquetados.

Estimar a distribuição posterior de `N`, o número de ursos no ambiente.

Para os anteriores, utilizar uma distribuição uniforme discreta de 50 a 500.

Ver [the PyMC3 documentation](https://docs.pymc.io/api/distributions/discrete.html) para a lista de distribuições discretas.

Nota: `HyperGeometric` foi adicionado ao PyMC3 após a versão 3.8, pelo que poderá ter de actualizar a sua instalação para fazer este exercício.

In [63]:
# Solution goes here

**Exercício:** Em <<_TheWeibullDistribution>> gerámos uma amostra de uma distribuição Weibull com $\lambda=3$ e $k=0,8$.

Depois utilizámos os dados para calcular uma aproximação em grelha da distribuição posterior desses parâmetros.

Agora vamos fazer o mesmo com o PyMC3.

Para os antecedentes, pode utilizar distribuições uniformes como fizemos em <<_SurvivalAnalysis>>, ou pode utilizar as distribuições `HalfNormal` fornecidas por PyMC3.

Nota: A classe `Weibull` em PyMC3 utiliza parâmetros diferentes de SciPy.  O parâmetro `alpha` em PyMC3 corresponde a $k$, e `beta` corresponde a $\lambda$.

Aqui estão novamente os dados:

In [64]:
data = [0.80497283, 2.11577082, 0.43308797, 0.10862644, 5.17334866,
       3.25745053, 3.05555883, 2.47401062, 0.05340806, 1.08386395]

In [65]:
# Solution goes here

**Exercício:** Em <<_ImprovingReadingAbility>> utilizámos dados de um teste de leitura para estimar os parâmetros de uma distribuição normal.

Fazer um modelo que define distribuições prévias uniformes para `mu` e `sigma` e utiliza os dados para estimar as suas distribuições posteriores.

Aqui estão novamente os dados.

In [66]:
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/drp_scores.csv')

In [67]:
import pandas as pd

df = pd.read_csv('drp_scores.csv', skiprows=21, delimiter='\t')
df.head()

Vou utilizar "grupo por grupo" para separar o grupo tratado do grupo de controlo.

In [68]:
grouped = df.groupby('Treatment')
responses = {}

for name, group in grouped:
    responses[name] = group['Response']

Agora estimar os parâmetros para o grupo tratado.

In [69]:
data = responses['Treated']

In [70]:
# Solution goes here

In [71]:
# Solution goes here

**Exercise:** Em <<_TheLincolnIndexProblem>> utilizámos um algoritmo de grelha para resolver o problema do Índice Lincoln, tal como apresentado por John D. Cook:

> "Suponha que tem um testador que encontra 20 bugs no seu programa. Deseja estimar quantos bugs existem realmente no programa. Sabe que existem pelo menos 20 bugs, e se tiver confiança suprema no seu testador, pode supor que existem cerca de 20 bugs. Mas talvez o seu testador não seja muito bom. Talvez haja centenas de bugs. Como pode ter alguma ideia de quantos insectos existem? Não há maneira de saber com um testador. Mas se tiver dois testadores, pode ter uma boa ideia, mesmo que não saiba quão habilidosos são os testadores".

Suponha que o primeiro verificador encontra 20 insectos, o segundo encontra 15, e eles

encontrar 3 em comum; utilizar PyMC3 para estimar o número de bugs.

Nota: Este exercício é mais difícil do que alguns dos anteriores.  Um dos desafios é que os dados incluem `k00`, o que depende de `N`:

```
k00 = N - num_seen
```

Portanto, temos de construir os dados como parte do modelo.

Para tal, podemos utilizar `pm.math.stack', o que faz uma matriz:

```
data = pm.math.stack((k00, k01, k10, k11))
```

Finalmente, poderá achar útil utilizar `pm.Multinomial'.

Vou utilizar a seguinte notação para os dados:

* k11 é o número de insectos encontrados por ambos os testadores,

* k10 é o número de insectos encontrados pelo primeiro testador mas não pelo segundo,

* k01 é o número de bugs encontrados pelo segundo testador mas não pelo primeiro, e

* k00 é o número desconhecido de bugs não descobertos.

Aqui estão os valores para todos menos `k00`:

In [72]:
k10 = 20 - 3
k01 = 15 - 3
k11 = 3

No total, foram descobertos 32 insectos:

In [73]:
num_seen = k01 + k10 + k11
num_seen

In [74]:
# Solution goes here

In [75]:
# Solution goes here

In [76]:
# Solution goes here